In [1]:

import jax
jax.config.update('jax_enable_x64',True)




In [2]:


import numpyro
from NSF import NeuralSpline1D
from flow import Normal,Flow,transform,Serial
from SkewNormalPlus import SkewNormalPlus as snp

In [3]:
from jax import random
import jax_cosmo as jc
from jax_cosmo import Cosmology, background
import numpy as onp
import jax.numpy as np
from jax import grad, jit, vmap, value_and_grad
from jax.scipy.stats import norm

In [4]:

@jit
def make_training_set(key,z_s,M0=-19.5,sigma_int=0.1,h=0.7324,Om0=0.28,w=-1,mu_cut=21,sigma_cut=0.01):




    cosmo_jax = Cosmology(Omega_c=Om0, h=h, w0=w, Omega_b=0, n_s= 0.96, sigma8=200000, Omega_k=0, wa=0)

    n_sne=len(z_s)

    d_s=background.transverse_comoving_distance(cosmo_jax, 1/(1+z_s))

    mu_s=5*np.log10((1+z_s)/h*d_s*1e6/10)

    m0_s=M0+mu_s

    m_s=random.normal(key, (len(m0_s),))*sigma_int+m0_s

    key, _ = random.split(key)

    p_s=norm.cdf(-m_s, loc=-mu_cut, scale=sigma_cut)

    sel_s=random.bernoulli(key, p=p_s)

    return m0_s,m_s,sel_s



def minmax_fit_and_scale(X):
  max= np.max(X,axis=0)
  min = np.min(X,axis=0)
  X_std = (X - min) / (max-min)
  return X_std,min,max


def minmax_scale(X,min,max):
  return (X - min) / (max - min)







In [5]:
from jax.example_libraries import stax, optimizers

from jax.example_libraries.stax import (Dense, Tanh, Flatten, Relu, LogSoftmax, Softmax, Exp,Sigmoid,Softplus,LeakyRelu)
'''
def network(rng,conditional_dim,out_dim, hidden_dim):
    init_fun,apply_fun=stax.serial(stax.Dense(hidden_dim),  Relu, stax.Dense(1024),Relu,stax.Dense(hidden_dim), Relu, stax.Dense(out_dim),)
    _, params = init_fun(rng, (conditional_dim,))

    return params,apply_fun
'''
def network(rng,conditional_dim,out_dim, hidden_dim):
    init_fun,apply_fun=stax.serial(stax.Dense(hidden_dim), Tanh, stax.Dense(1024),Tanh,stax.Dense(hidden_dim), Tanh, stax.Dense(out_dim),)
    _, params = init_fun(rng, (conditional_dim,))
    return params,apply_fun

rng, flow_rng = random.split(random.PRNGKey(43))

init_fun = Flow(Serial(*(NeuralSpline1D(network,hidden_dim=128,K=5,B=3),)*2),prior=Normal())

params, log_pdf, sample = init_fun(flow_rng,2 )

In [6]:
opt_init, opt_update, get_params = optimizers.adam(step_size=1e-4)
opt_state = opt_init(params)

In [7]:
def loss_fn(params, inputs):
    return -log_pdf(params, inputs).mean()

@jit
def step(i, opt_state, inputs):
    params = get_params(opt_state)

    loss,gradients = value_and_grad(loss_fn)(params,inputs)
    return loss, opt_update(i, gradients, opt_state)

def minmax_unscale(X,min,max):
  return X * (max - min) + min


In [8]:
from tqdm.notebook import trange
import itertools
import numpy.random as npr
itercount = itertools.count()
sample_batch=1000
batch_size=1000
import logging
itercount = itertools.count()
from IPython.display import clear_output




for epoch in range(1000):
    

  X = np.empty((0,3))
 
  for s_epoch in range(20):
    
      permute_rng, rng = random.split(rng)  
      sigma_int =   random.uniform(rng,(1,))

  
      sigma_cut=0.01
      mu_cut= 21

      permute_rng, rng = random.split(rng)


      mu=random.uniform(rng,(10,))*(21+2*sigma_int[0]-20.5) + 20.5


      s = snp(m_int=mu,sigma_int=sigma_int,m_cut=mu_cut,sigma_cut=sigma_cut)

      permute_rng, rng = random.split(rng)
      

      samps=s.sample(rng,(5,10))
        
      permute_rng, rng = random.split(rng)



      theta = np.column_stack((np.repeat(mu,5).T.reshape(50,1),np.repeat(sigma_int,50).reshape(50,1)))
    

        
      X_ = np.column_stack((samps.T.reshape(50,1),theta))
    
      X = np.append(X,X_,axis=0)
        
  if epoch == 0:
      X,min,max=minmax_fit_and_scale(X)

    
  else:
      X  = minmax_scale(X,min,max)



  print(X)
  loss,opt_state = step(next(itercount), opt_state, X)
  params = get_params(opt_state)
  print('epoch: ',epoch,' loss: ', loss)

      
  
  if epoch % 100 == 0 :
        from matplotlib import pyplot as plt
        c=['red','green','blue','yellow','pink']
        mm = np.array([20.7,20.8,20.9,21,21.1])
        bins=np.linspace(20.3,21.1,100)
        no_samps = 50000


        plt.figure(figsize=(12,6))



        for i,m in enumerate(mm):
            

            samp=sample(rng, params,minmax_scale(np.column_stack((np.repeat(np.array([m]),no_samps).reshape(no_samps,1),np.array([0.1]*no_samps).reshape(no_samps,1))),min[1:],max[1:]),no_samps)
            samp = minmax_unscale(samp[:,0],min[0],max[0])

            plt.hist(samp,density=True,bins=bins,color=c[i],histtype='step',label='$m_0=$'+str(m)+',$\sigma_{int}=0.1$,$m_{cut}=$'+str(mu_cut)+',$\sigma_{cut}=$'+str(sigma_cut))
            s=snp(np.array([m]),np.array([0.1]),21,0.01)
            plt.plot(bins,np.exp(s.log_prob(bins)),color=c[i])
    

    
        plt.legend(loc='center left',frameon=False,fontsize=12)
        plt.xlabel('$\hat{m}_s$',fontsize=11)
        plt.ylabel('$p(\hat{m}_s|m_0,\sigma_{int},m_{cut},\sigma_{cut})$',fontsize=12)
        plt.title('Normalising Flow',fontsize=12)

        plt.show()
    

[[0.88080944 0.02687101 0.98478856]
 [0.30048246 0.02687101 0.98478856]
 [0.62639956 0.02687101 0.98478856]
 ...
 [0.7615477  0.03737395 0.1577597 ]
 [0.86385331 0.03737395 0.1577597 ]
 [0.68446517 0.03737395 0.1577597 ]]


KeyboardInterrupt: 

In [ ]:


for i in range(X.shape[0]):
    print(X[i,:])

In [ ]:
rng, flow_rng = random.split(random.PRNGKey(0))


def minmax_unscale(X,min,max):
  return X * (max - min) + min




no_samps = 100000

samp=sample(rng, params,minmax_scale(np.repeat(np.array([21.1]),no_samps).reshape(no_samps,1),min[1:],max[1:]),no_samps)

print(samp)

samp = minmax_unscale(samp[:,0],min[0],max[0])



In [ ]:
print(samp.shape)
print(min)
print(max[1:])


In [ ]:
from matplotlib import pyplot as plt
plt.hist(samp,bins=100)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
c=['red','green','blue','yellow','pink']
mm = np.array([20.7,20.8,20.9,21,21.1])
bins=np.linspace(20.3,21.1,100)
no_samps = 100000


plt.figure(figsize=(12,6))



for i,m in enumerate(mm):
    
    print(minmax_scale(np.repeat(np.array([m]),no_samps).reshape(no_samps,1),min[1:],max[1:]))
    samp=sample(rng, params,minmax_scale(np.repeat(np.array([m]),no_samps).reshape(no_samps,1),min[1:],max[1:]),no_samps)
    samp = minmax_unscale(samp[:,0],min[0],max[0])

    plt.hist(samp,density=True,bins=bins,color=c[i],histtype='step',label='$m_0=$'+str(m)+',$\sigma_{int}=$'+str(sigma_int)+',$m_{cut}=$'+str(mu_cut)+',$\sigma_{cut}=$'+str(sigma_cut))
    s=snp(np.array([m]),np.array([0.1]),21,0.01)
    plt.plot(bins,np.exp(s.log_prob(bins)),color=c[i])
    
    print(np.min(samp),np.max(samp))

    #plt.hist(s.sample(rng,(1000,1)).reshape(1000,),density=True,histtype='step')
    

plt.legend(loc='center left',frameon=False,fontsize=12)
plt.xlabel('$\hat{m}_s$',fontsize=11)
plt.ylabel('$p(\hat{m}_s|m_0,\sigma_{int},m_{cut},\sigma_{cut})$',fontsize=12)
plt.title('Normalising Flow',fontsize=12)

plt.show()

In [ ]:
plt.hist(minmax_scale(samp,20,22),bins=minmax_scale(np.linspace(20,22,100),20,22))
plt.show()

In [ ]:
a = np.arange(10)

print(np.repeat(a,10))

In [ ]:
bins = np.linspace(2.75,3.25,100)

plt.hist(samp,density=True,bins=bins)

In [ ]:
plt.hist(samp,bins=100)

In [ ]:
bs=np.arange(10)

XX= np.column_stack((bs,bs))


print(np.repeat(bs.reshape(10,1),5,axis=1).reshape(50,))


In [ ]:
print(X)

In [ ]:

permute_rng, rng = random.split(rng)

s = snp(m_int=np.array([20.99]),sigma_int=0.1,m_cut=21,sigma_cut=0.01)
xxx=s.sample(rng,(10000,1)).reshape(10000,)
rr= np.linspace(np.min(xxx),np.max(xxx),100)
plt.plot(rr,np.exp(s.log_prob(rr)))
plt.hist(xxx,bins=100,density=True)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
sigma_int=0.1
sigma_cut=0.01
mu_cut= 21

permute_rng, rng = random.split(random.PRNGKey(0))


sigma_int=0.1
sigma_cut=0.01
mu_cut= 21

mu=random.uniform(rng,(10,)) + 20.5

permute_rng, rng = random.split(rng)


s = snp(m_int=mu,sigma_int=sigma_int,m_cut=mu_cut,sigma_cut=sigma_cut)



samps=s.sample(rng,(1000,10))




X = np.column_stack((samps.T.reshape(10000,),np.repeat(mu.reshape(10,1),1000,axis=1).reshape(10000,)))







for i in range(10):
  xxx=X[i*1000:(i+1)*1000,0]
  plt.hist(xxx,bins=100,density=True)
  s = snp(m_int=np.array([X[i*1000,1]]),sigma_int=0.1,m_cut=21,sigma_cut=0.01)
  rr= np.linspace(np.min(xxx),np.max(xxx),100)
  plt.plot(rr,np.exp(s.log_prob(rr)))
  plt.show()



In [ ]:
print